In [1]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

Using TensorFlow backend.


In [2]:
from string import punctuation

In [3]:
numpy.random.seed(7)
# load the dataset but only keep the top n words, zero the rest
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 599dadb1135973df5b59232a0e9a887c so we will re-download the data.
17465344/17464789 [==============================] - 36s 2us/step


In [4]:
X_train.shape

(25000,)

In [22]:
X_test.shape

(25000,)

In [90]:
(X_train[:2])

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 2, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 2, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 2, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 2, 19, 178, 32]),
       list([1, 194, 1153, 194, 2, 78, 228, 5, 6, 1463, 4369,

In [6]:
y_test[:5]

array([0, 1, 1, 0, 1])

In [3]:
#load the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as  plt
#import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [4]:
# load the data
yelp=pd.read_csv('yelp.csv')

In [5]:
#Check the header 
yelp.head(n=2)

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0


In [6]:
#convert the problem into binary classification
yelp.loc[yelp.stars>3 , 'Sentiment']='Positive'

In [7]:
yelp.loc[yelp.stars<=3 ,'Sentiment']='Negative'

In [8]:
# recheck the header of the file
yelp.head(n=1)

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny,Sentiment
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0,Positive


In [9]:
#Get the relevant columns necessary for classification leaving the rest : reviews and the stars
reviews = yelp.text.values

In [10]:
labels = yelp.Sentiment.values


In [11]:
#Integer encode the labels
labels = np.array([1 if s=='POSITIVE' else 0 for s in labels ])

In [12]:
labels[:5]

array([0, 0, 0, 0, 0])

In [18]:
#convert the reviews in the same format of that of keras
review_cleaned =[]
for i in range(len(reviews)):
    
    review_cleaned.append(''.join([c.replace('\n' , ' ').lower() for c in reviews[i] if c not in punctuation ]))

In [19]:
review_cleaned[:5]

['my wife took me here on my birthday for breakfast and it was excellent  the weather was perfect which made sitting outside overlooking their grounds an absolute pleasure  our waitress was excellent and our food arrived quickly on the semibusy saturday morning  it looked like the place fills up pretty quickly so the earlier you get here the better  do yourself a favor and get their bloody mary  it was phenomenal and simply the best ive ever had  im pretty sure they only use ingredients from their garden and blend them fresh when you order it  it was amazing  while everything on the menu looks excellent i had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious  it came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete  it was the best toast ive ever had  anyway i cant wait to go back',
 'i have no idea why some people give bad reviews about this place it goes to show you you can please everyone they are probabl

In [20]:
print('Before cleaning :' , reviews[:2])
print('\n')
print('After cleaning', review_cleaned[:2])

Before cleaning : ['My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.\n\nDo yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I\'ve ever had.  I\'m pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.\n\nWhile EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I\'ve ever had.\n\nAnyway, I can\'t wait to go back!'
 'I have no idea why some people give bad reviews about this place. It goes to 

In [62]:
#I need to encode the document for this I need to create a vocabulary
vocab_n = set(' '.join(review_cleaned).split())

In [63]:
(vocab_n)

{'intimately',
 'embittered',
 'dogfriendly',
 'biergarten',
 'borderline',
 'higher',
 'favoritos',
 'westgate',
 'nachosbad',
 'formulalike',
 'decomissioned',
 'girlfriends',
 'flaked',
 'basketbut',
 'kilmlike',
 'authenticstyle',
 'halfdead',
 'personalityvery',
 'cheezecake',
 'timepieces',
 'tryouts',
 'advised',
 'cinch',
 'gentleman',
 'larry',
 'apothecary',
 'tat',
 'westside',
 'soupthought',
 'ddelish',
 'argues',
 'tragic',
 'teenscant',
 'tortillasand',
 'migrating',
 '1214',
 'rumored',
 'infrastructure',
 'schoolnot',
 'boozers',
 'vinaigrette',
 'importantlygo',
 'uttered',
 'souvenirs',
 'jive',
 'secondary',
 'remortgage',
 'tel',
 'unsolicited',
 'allison',
 'pricks',
 'unbearably',
 'sangria',
 '675',
 'splayed',
 'hospitality',
 'consignment',
 'dishwashers',
 'suffer',
 'bucked',
 'mountains',
 'guiness',
 'houseswell',
 'starting',
 'icecold',
 'shed',
 'selects',
 'greatwe',
 'bursting',
 'ventilation',
 'storedont',
 'michelada',
 'struggled',
 'dentist',
 'e

In [64]:
#Create a dictionary for the vocab.
vocab_to_int ={word:i for i , word in enumerate(vocab_n)}

In [65]:
print(vocab_to_int)

{'intimately': 0, 'embittered': 1, 'dogfriendly': 2, 'biergarten': 3, 'borderline': 4, 'higher': 5, 'favoritos': 6, 'westgate': 7, 'nachosbad': 8, 'formulalike': 9, 'decomissioned': 10, 'girlfriends': 11, 'flaked': 12, 'basketbut': 13, 'kilmlike': 14, 'authenticstyle': 15, 'halfdead': 16, 'personalityvery': 17, 'cheezecake': 18, 'timepieces': 19, 'tryouts': 20, 'advised': 21, 'cinch': 22, 'gentleman': 23, 'larry': 24, 'apothecary': 25, 'tat': 26, 'westside': 27, 'soupthought': 28, 'ddelish': 29, 'argues': 30, 'tragic': 31, 'teenscant': 32, 'tortillasand': 33, 'migrating': 34, '1214': 35, 'rumored': 36, 'infrastructure': 37, 'schoolnot': 38, 'boozers': 39, 'vinaigrette': 40, 'importantlygo': 41, 'uttered': 42, 'souvenirs': 43, 'jive': 44, 'secondary': 45, 'remortgage': 46, 'tel': 47, 'unsolicited': 48, 'allison': 49, 'pricks': 50, 'unbearably': 51, 'sangria': 52, '675': 53, 'splayed': 54, 'hospitality': 55, 'consignment': 56, 'dishwashers': 57, 'suffer': 58, 'bucked': 59, 'mountains': 6

In [174]:
'''make all reviews numbers using 
the vocab that I created just now...'''
def reviews_to_integers(reviews):
    reviews_to_int= []
    for i in range(len(reviews)):
        to_int=[vocab_to_int[word] for word in reviews[i].split()]
        reviews_to_int.append(to_int)
    return  reviews_to_int
reviews_to_int =reviews_to_integers(review_cleaned)

In [176]:
#make it numpy compatible...
reviews_to_int=np.array(reviews_to_int)

In [177]:
#max(reviews_to_int)

In [178]:
X_train=reviews_to_int

In [179]:
y_train= labels

In [180]:
#Check the shape of the data ... pretty similar to keras dataset...
y_train.shape

(10000,)

In [181]:
X_train.shape

(10000,)

In [182]:
# I am considering the length of a review to 5 (limited computing resource)
# any reviews less than size of 5 will be padded with '0' and more than five will be truncated to 5.
max_review_len=5

In [183]:
#Do padding-- any reviews less than size of 5 will be padded with '0' and more than five will be truncated to 5.
X_train = sequence.pad_sequences(X_train ,maxlen=max_review_len)

In [184]:
# assuming the vector dimension to be 16 and the top words = total words in the vocab , as the vocab is pretty small.
embedding_vector_length = 16
top_words=36500

In [185]:
'''define a LSTM model
embedding-->LSTM-->Classification(binary)'''
model = Sequential()

In [186]:
model.add(Embedding(top_words , embedding_vector_length, input_length=max_review_len))

In [187]:
model.add(LSTM(100))

In [188]:
model.add(Dense(1,activation='sigmoid'))

In [189]:
model.compile(loss='binary_crossentropy', optimizer ='adam',metrics= ['accuracy'])

In [190]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 5, 16)             584000    
_________________________________________________________________
lstm_7 (LSTM)                (None, 100)               46800     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 101       
Total params: 630,901
Trainable params: 630,901
Non-trainable params: 0
_________________________________________________________________
None


In [191]:
model.fit(X_train , y_train , epochs=2,batch_size=32)

Epoch 1/2
10000/10000 [==============================] - 19s 2ms/step - loss: 0.0486 - acc: 0.9986
Epoch 2/2
10000/10000 [==============================] - 17s 2ms/step - loss: 5.6984e-06 - acc: 1.0000


In [192]:
# To Do : do a train test split?
# check what I did was even correct???
# Get feed back.
# Learn about better ways of pre-processing my data?(Stackoverflow , appliedai.com)
#Get more sleep!!!